In [1]:
import os
import matplotlib
import numpy as np
import pandas as pd
import csv
import datetime
from collections import Counter

pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', None) 
pd.options.plotting.backend = "plotly"

In [2]:
directory = "API-logs/"

### Code Carbon logs

In [3]:
carbondf = pd.DataFrame(columns = ['date', 'timestamp', 'RAM energy', 'GPU energy', 'CPU energy', 'Energy consumed'])

In [4]:
ins=[]
with open(directory+'carbon.log' , 'r') as f:
    lines = f.readlines() # readlines creates a list of the lines
    for line in lines:
        if 'Sep' in line:
            ins.append(lines.index(line))
len(lines)

143138

In [5]:
alldates, sep22, sep23, sep24, sep25, sep26= [], [], [], [], [], []
sep22 = lines[ins[0]:ins[1]]
print(len(sep22))
alldates.append(sep22)
sep23 = lines[ins[1]:ins[2]]
print(len(sep23))
alldates.append(sep23)
sep24 = lines[ins[2]:ins[3]]
alldates.append(sep24)
print(len(sep24))
sep25 = lines[ins[3]:ins[4]]
alldates.append(sep25)
print(len(sep25))
sep26 = lines[ins[4]:len(lines)]
alldates.append(sep26)
print(len(sep26))

13408
36577
36573
36577
20003


In [6]:
count=0
for day in alldates: 
    date= day[0].replace('\n','')
    print(date)
    print(len(day))
    for i in range(1, len(day), 4):
        for line in day[i:i + 4]:
            try:
                time = line.split('@')[1].split(']')[0].strip()
            except:
                continue
            if 'RAM :' in line: 
                ram_nrj = line.split('Energy consumed for RAM :')[1].split('RAM')[0].replace('kWh.','')
            elif 'GPUs'in line:
                gpu_nrj = line.split('Energy consumed for all GPUs :')[1].split('All GPUs')[0].replace('kWh.','')
            elif 'CPUs' in line:
                cpu_nrj= line.split('Energy consumed for all CPUs :')[1].split('All CPUs')[0].replace('kWh.','')
            elif ' electricity used since the begining.' in line:
                total_nrj = line.split(' electricity used since the begining.')[0].split(']')[1].replace(' of','').replace('kWh','')
        carbondf.loc[count] = [date, time, ram_nrj, gpu_nrj, cpu_nrj, total_nrj]
        count+=1

22/Sep/2022
13408
23/Sep/2022
36577
24/Sep/2022
36573
25/Sep/2022
36577
26/Sep/2022
20003


In [7]:
carbondf["Energy consumed"] = pd.to_numeric(carbondf["Energy consumed"])

In [8]:
carbondf.shape

(35784, 6)

In [9]:
carbondf.date.value_counts()

23/Sep/2022    9144
25/Sep/2022    9144
24/Sep/2022    9143
26/Sep/2022    5001
22/Sep/2022    3352
Name: date, dtype: int64

In [10]:
#carbondf = carbondf.sort_values('Energy consumed')
#carbondf = carbondf.reset_index(drop=True)

In [11]:
carbondf.head()

,date,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed
0,22/Sep/2022,15:12:02,0.001395,0.006703,0.000125,0.008222
1,22/Sep/2022,15:12:12,0.002708,0.013103,0.000243,0.016054
2,22/Sep/2022,15:12:22,0.004022,0.021320,0.000361,0.025703
3,22/Sep/2022,15:12:32,0.005333,0.024947,0.000478,0.030758
4,22/Sep/2022,15:12:42,0.006658,0.028631,0.000597,0.035886


In [12]:
carbondf['time'] = carbondf['date'] + ' ' + carbondf['timestamp'].astype(str)

In [13]:
carbondf['time'] = pd.to_datetime(carbondf['time'] , format= "%d/%b/%Y %H:%M:%S")

In [55]:
carbondf.head()

,date,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed,time
0,22/Sep/2022,15:12:02,0.001395,0.006703,0.000125,0.008222,2022-09-22 15:12:02
1,22/Sep/2022,15:12:12,0.002708,0.013103,0.000243,0.016054,2022-09-22 15:12:12
2,22/Sep/2022,15:12:22,0.004022,0.021320,0.000361,0.025703,2022-09-22 15:12:22
3,22/Sep/2022,15:12:32,0.005333,0.024947,0.000478,0.030758,2022-09-22 15:12:32
4,22/Sep/2022,15:12:42,0.006658,0.028631,0.000597,0.035886,2022-09-22 15:12:42


## Energy consumption by 15 min

In [74]:
carbonhourdf = carbondf.groupby(pd.Grouper(key = 'time', freq='15min')).last()
carbonhourdf.head()

,date,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed
time,,,,,,
2022-09-22 15:00:00,22/Sep/2022,15:14:52,0.023764,0.096576,0.002132,0.122471
2022-09-22 15:15:00,22/Sep/2022,15:29:52,0.142635,0.615689,0.012757,0.763339
2022-09-22 15:30:00,22/Sep/2022,15:44:52,0.261070,1.247989,0.023264,1.524364
2022-09-22 15:45:00,22/Sep/2022,15:59:52,0.379544,1.836246,0.033890,2.248372
2022-09-22 16:00:00,22/Sep/2022,16:14:52,0.497802,2.417497,0.044633,2.959932


In [75]:
carbonhourdf['Energy by interval'] = carbonhourdf["Energy consumed"].diff()

In [76]:
carbonhourdf.head()

,date,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed,Energy by interval
time,,,,,,,
2022-09-22 15:00:00,22/Sep/2022,15:14:52,0.023764,0.096576,0.002132,0.122471,NaN
2022-09-22 15:15:00,22/Sep/2022,15:29:52,0.142635,0.615689,0.012757,0.763339,0.640868
2022-09-22 15:30:00,22/Sep/2022,15:44:52,0.261070,1.247989,0.023264,1.524364,0.761025
2022-09-22 15:45:00,22/Sep/2022,15:59:52,0.379544,1.836246,0.033890,2.248372,0.724008
2022-09-22 16:00:00,22/Sep/2022,16:14:52,0.497802,2.417497,0.044633,2.959932,0.711560


In [77]:
carbonhourdf = carbonhourdf.iloc[1:-1 , :]
carbonhourdf.shape

(375, 7)

In [78]:
carbonhourdf.tail()

,date,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed,Energy by interval
time,,,,,,,
2022-09-26 11:45:00,26/Sep/2022,11:59:57,44.141909,148.180237,3.943960,196.266107,0.503662
2022-09-26 12:00:00,26/Sep/2022,12:14:57,44.261019,148.551300,3.954586,196.762001,0.495894
2022-09-26 12:15:00,26/Sep/2022,12:29:57,44.379839,148.984825,3.965095,197.319061,0.557060
2022-09-26 12:30:00,26/Sep/2022,12:44:57,44.498630,149.395452,3.975719,197.868487,0.549426
2022-09-26 12:45:00,26/Sep/2022,12:59:58,44.617507,149.795291,3.986463,198.399261,0.530774


### Server logs

In [18]:
with open(directory+'time-server.log','r') as f:
    lines = f.readlines() # readlines creates a list of the lines
len(lines)

54481

In [19]:
requestdf = pd.DataFrame(columns = ['date', 'requests'])

In [20]:
import re
count=0
for line in lines:
    if "POST /generate HTTP/1.1" in line:
        time = line.split('[')[1].split(']')[0]
        code = line.split("POST /generate HTTP/1.1")[1]
        code= re.sub("[^0-9]", "", code)
        code= code.replace('0400','400')
        requestdf.loc[count] = [time, code]
        count+=1

In [21]:
requestdf['date'] = pd.to_datetime(requestdf['date'] , format= "%d/%b/%Y %H:%M:%S")

In [40]:
requestdf['requests'] = pd.to_numeric(requestdf['requests'])

In [61]:
requestdf.request.value_counts()

200    53332
400     1149
Name: request, dtype: int64

200 = OK we answered

400 = used bad request

503 = refused because no more room in the queue

### Number of requests per 15 minutes

In [79]:
requesthourdf = requestdf.groupby(pd.Grouper(key = 'date', freq='15min')).count()
requesthourdf = requesthourdf.iloc[:-1 , :]
requesthourdf.tail()

,request
date,
2022-09-26 11:45:00,140
2022-09-26 12:00:00,115
2022-09-26 12:15:00,206
2022-09-26 12:30:00,209
2022-09-26 12:45:00,142


In [80]:
alllogdf = pd.concat([carbonhourdf,requesthourdf], axis=1)
alllogdf['date']= alllogdf.index
alllogdf.head()

,date,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed,Energy by interval,request
2022-09-22 15:15:00,2022-09-22 15:15:00,15:29:52,0.142635,0.615689,0.012757,0.763339,0.640868,322
2022-09-22 15:30:00,2022-09-22 15:30:00,15:44:52,0.261070,1.247989,0.023264,1.524364,0.761025,315
2022-09-22 15:45:00,2022-09-22 15:45:00,15:59:52,0.379544,1.836246,0.033890,2.248372,0.724008,701
2022-09-22 16:00:00,2022-09-22 16:00:00,16:14:52,0.497802,2.417497,0.044633,2.959932,0.711560,571
2022-09-22 16:15:00,2022-09-22 16:15:00,16:29:52,0.616152,2.986218,0.055257,3.651571,0.691639,355


In [86]:
alllogdf[alllogdf['request']== 701]

,date,timestamp,RAM energy,GPU energy,CPU energy,Energy consumed,Energy by interval,request,normalized energy,normalized requests,log energy,log requests
2022-09-22 15:45:00,2022-09-22 15:45:00,15:59:52,0.379544,1.836246,0.033890,2.248372,0.724008,701,0.951359,1.0,-0.140257,2.845718


In [82]:
alllogdf["normalized energy"] = alllogdf['Energy by interval'] / alllogdf['Energy by interval'].max()
alllogdf["normalized requests"] = alllogdf["request"] / alllogdf["request"].max()

In [83]:
alllogdf['log energy'] = np.log10(alllogdf['Energy by interval'] )
alllogdf['log requests'] = np.log10(alllogdf['request'] )

### Plotting raw values

In [84]:
alllogdf.plot(x="date", y=["Energy by interval", "request"], kind="line")

In [85]:
alllogdf.plot(x="request", y="Energy by interval", kind="scatter")

### Plotting log values

In [49]:
alllogdf.plot(x="date", y=["log energy", "log requests"], kind="line")

### Plotting normalized values

In [45]:
alllogdf.plot(x="date", y=["normalized energy", "normalized requests"], kind="line")